## SRP366684 / GSE199780

**paper:** [PMID 36642281](https://www.sciencedirect.com/science/article/pii/S0888754323000083?via%3Dihub#s0010) - RNA-seq analysis of the active chick embryo chorioallantoic membrane reveals genes that encode proteins assigned to ion transport and innate immunity
- also [PMID 39263233](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11388263/) - RNA-seq dataset of the chorioallantoic membrane of male and female chicken embryos, after 11 and 15 days of incubation

**date, curator:** 2024-09-19, Sara Carsanaro

In [58]:
experiment_id = "SRP366684"

path_to_create_exp_script = "/Users/scarsana/Desktop/git/scRNA-Seq/scripts/Create_ExpLib_tables.py" 
experiment_type = "bulk"

path_to_output_main = "/Users/scarsana/Desktop/git/expression-annotations/Notebooks/bulk/" 
path_to_output = "{}{}/".format(path_to_output_main, experiment_id)
library_path_from_script = "{}RNASeqLibrary_{}.tsv".format(path_to_output, experiment_id)
experiment_path_from_script = "{}RNASeqExperiment_{}.tsv".format(path_to_output, experiment_id)
library_to_add_path = "{}complete_RNASeqLibrary_{}.tsv".format(path_to_output, experiment_id)
experiment_to_add_path = "{}complete_RNASeqExperiment_{}.tsv".format(path_to_output, experiment_id)
script_file = "{}.ipynb".format(experiment_id)
#commit_message_py = '"adding annotation files for {} to notebook folder"'.format(experiment_id)
commit_message_exp = '"adding annotated bulk experiment {} and adding annotation files to notebook folder"'.format(experiment_id)

## for desktop testing
#path_to_output_desktop = "/Users/scarsana/Desktop/annotate/redo/" 
#path_to_output = "{}{}/".format(path_to_output_desktop, experiment_id)

## to add to git
path_to_git_annotations = "/Users/scarsana/Desktop/git/expression-annotations/RNA_Seq/"
git_library_path = "{}RNASeqLibrary.tsv".format(path_to_git_annotations)
git_experiment_path = "{}RNASeqExperiment.tsv".format(path_to_git_annotations)


In [41]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import os
import csv

# displays df with the scrollbar next to the DataFrame
def display_df(df):
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    display(HTML("<div style='height: 200px; overflow: auto; width: fit-content'>" +
        df.style.to_html(index=False) + "</div>"))

# function that compares two columns in a dataframe and tells you which ones are not equal (case insensitive)
def compare_columns(df, col1, col2, return_col):
    compare_return = df[col1].str.lower() != df[col2].str.lower()  
    df.loc[compare_return, return_col] 
    if not any(compare_return):
        print("The two columns are equal (case insensitive)")
    else:
        print("The following rows are not equal: ")
        print(df.loc[compare_return, return_col])


def update_format(path):
    with open(path, 'r') as file:
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace("\t\"\"", "\t")
    # Write the file out again
    with open(path, 'w') as file:
        file.write(filedata)

def dup_check(df, column):
    duplicateCheck = df.duplicated(subset=[column], keep=False)
    if duplicateCheck.unique() == False:
        print("no duplicate values in " + column)
    elif duplicateCheck.unique() == True and column != '#libraryId':
        print(df[duplicateCheck].loc[:,['#libraryId', column]])
    elif duplicateCheck.unique() == True and column == '#libraryId':
        print(df[duplicateCheck].loc[:,['#libraryId']])

In [43]:
dup_check(library, "SRSId")
#duplicateCheck = library.duplicated(subset=["anatId"], keep=False)
#duplicateCheck.unique() 
#library[duplicateCheck].loc[:,['#libraryId', "anatId"]]

no duplicate values in SRSId


### script

In [3]:
! python3 $path_to_create_exp_script $experiment_id $path_to_output $experiment_type

/Users/scarsana/Desktop/git/scRNA-Seq/scripts/Create_ExpLib_tables.py:97: SyntaxWarning: invalid escape sequence '\('
  all_protoc = [w.replace('(', '\(') for w in all_protoc]
/Users/scarsana/Desktop/git/scRNA-Seq/scripts/Create_ExpLib_tables.py:98: SyntaxWarning: invalid escape sequence '\)'
  all_protoc = [w.replace(')', '\)') for w in all_protoc] 
Be patient, it may take a few minutes.
0it [00:00, ?it/s]
5 samples dont have attributes, try to find them somewhere else
100%|█████████████████████████████████████████████| 5/5 [00:08<00:00,  1.70s/it]
0 samples dont have attributes


### library annnotations

In [4]:
library = pd.read_csv(library_path_from_script, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,,,,female,Ross 308 broiler,,9031,,,,,,CAM15F_102,SAMN27103190,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,,,,male,Ross 308 broiler,,9031,,,,,,CAM15M_100,SAMN27103191,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,,,,male,Ross 308 broiler,,9031,,,,,,CAM15M_98,SAMN27103192,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,,,,female,Ross 308 broiler,,9031,,,,,,CAM15F_95,SAMN27103193,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
4,SRX14667189,SRP366684,Illumina NovaSeq 6000,SRS12431810,,,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984238,Chorioallantoic membrane,Prehatch : 15 days of incubation,,,,male,Ross 308 broiler,,9031,,,,,,CAM15M_93,SAMN27103194,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendation

#### anatomical entity
chicken chorioallantoic membranes 

In [5]:

library.loc[:,'anatId'] = 'UBERON:0005335'
library.loc[:,'anatName'] = 'chorioallantoic membrane'
# perfect match, missing child term, other
library.loc[:,'anatAnnotationStatus'] = 'perfect match'
# partial sampling, full sampling, not documented
library.loc[:,'anatBiologicalStatus'] = 'not documented'

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,,female,Ross 308 broiler,,9031,,,,,,CAM15F_102,SAMN27103190,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,,male,Ross 308 broiler,,9031,,,,,,CAM15M_100,SAMN27103191,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,,male,Ross 308 broiler,,9031,,,,,,CAM15M_98,SAMN27103192,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,,female,Ross 308 broiler,,9031,,,,,,CAM15F_95,SAMN27103193,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
4,SRX14667189,SRP366684,Illumina NovaSeq 6000,SRS12431810,UBERON:0005335,chorioallantoic membrane,,,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984238,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,,male,Ross 308 broiler,,9031,,,,,,CAM15M_93,SAMN27103194,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentratio

#### stage
15 days of incubation: 
- Hamburger Hamilton stage 41
- Usually obtained after 15.0 days of incubation.

11 days of incubation: 
- Hamburger Hamilton stage 37
- Usually obtained after 11.0 days of incubation.

In [6]:
# conditional (based off infoStage)
library.loc[library["infoStage"] == "Prehatch : 15 days of incubation", "stageId"] = "GgalDv:0000055"
library.loc[library["infoStage"] == "Prehatch : 15 days of incubation", "stageName"] = "Hamburger Hamilton stage 41"
# perfect match, missing child term, other
library.loc[library["infoStage"] == "Prehatch : 15 days of incubation", "stageAnnotationStatus"] = "perfect match"

# conditional (based off infoStage)
library.loc[library["infoStage"] == "Prehatch : 11 days of incubation", "stageId"] = "GgalDv:0000051"
library.loc[library["infoStage"] == "Prehatch : 11 days of incubation", "stageName"] = "Hamburger Hamilton stage 37"
# perfect match, missing child term, other
library.loc[library["infoStage"] == "Prehatch : 11 days of incubation", "stageAnnotationStatus"] = "perfect match"

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,female,Ross 308 broiler,,9031,,,,,,CAM15F_102,SAMN27103190,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,male,Ross 308 broiler,,9031,,,,,,CAM15M_100,SAMN27103191,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,male,Ross 308 broiler,,9031,,,,,,CAM15M_98,SAMN27103192,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,female,Ross 308 broiler,,9031,,,,,,CAM15F_95,SAMN27103193,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
4,SRX14667189,SRP366684,Illumina NovaSeq 6000,SRS12431810,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GS

#### sex, strain, genotype, speciesId
- uniprot [strain list](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/strains)
- uniprot [species list](https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/docs/speclist)
- bgee [strain mapping](https://gitlab.sib.swiss/Bgee/expression-annotations/-/tree/develop/Strains?ref_type=heads)


strain from paper: Ross 308 broiler hens
- Ross and Broiler are accepted strains per uniprot
- our strain mapping has Ross 308 mapping to Ross strain

no genotype given

species ID 9031 is appropriate for Gallus gallus (chicken)

In [7]:
library.loc[library["sex"] == "male", "sex"] = "M"
library.loc[library["sex"] == "female", "sex"] = "F"

library.loc[:,'strain'] = 'Ross 308'

#library.loc[:,'genotype'] = ''

#library.loc[:,'speciesId'] = ''

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,,,,,,CAM15F_102,SAMN27103190,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,,,,,,CAM15M_100,SAMN27103191,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,,,,,,CAM15M_98,SAMN27103192,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,,,,,,CAM15F_95,SAMN27103193,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
4,SRX14667189,SRP366684,Illumina NovaSeq 6000,SRS12431810,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984238,Chorioallantoic membrane,Prehatch : 15 

#### protocol
see [bulk kits](https://gitlab.sib.swiss/Bgee/scRNA-Seq/-/blob/main/scripts/bulk_kits.csv) for some common protocols

NEB Next® Ultra™ RNA Library Prep Kit per paper

In [8]:
# making these variables because we use them again in the experiment file
my_protocol = 'NEBNext Ultra RNA Library Prep Kit'
# full_length or 3'
my_protocolType = 'full_length'

library.loc[:,'protocol'] = my_protocol
library.loc[:,'protocolType'] = my_protocolType
# polyA, ribo-minus, miRNA, lncRNA, circRNA
library.loc[:,'RNASelection'] = 'polyA'

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_102,SAMN27103190,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_100,SAMN27103191,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_98,SAMN27103192,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_95,SAMN27103193,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
4,SRX14667189,SRP366684,Illumina NovaSeq 6

#### globin, replicates

In [45]:
# check for duplicate SRSId values
dup_check(library, "SRSId")

no duplicate values in SRSId


In [46]:
#library.loc[:,'globin_reduction'] = 'Y'

# replicates
#library.loc[library["#libraryId"] == "old", "replicate"] = "1"
#library.loc[library["#libraryId"] in ["one", "two"], "replicate"] = "1"


# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_102,SAMN27103190,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_100,SAMN27103191,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_98,SAMN27103192,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_95,SAMN27103193,,,,,,,,,2024-9-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
4,SRX14667189,SRP366684,Illumina NovaSeq 6

#### sample age, pato, physiological status

In [ ]:
#library.loc[:,'sampleAge_value'] = ''
#library.loc[:,'sampleAge_unit'] = ''

# ex. castrated male
#library.loc[:,'PATOid'] = ''
#library.loc[:,'PATOname'] = ''

# ex. castrated, pregnant, pre-smoltification, post-smoltification, laying eggs
#library.loc[:,'physiologicalStatus'] = ''

# view
display_df(library)

#### condition

In [ ]:
# ex. control, diet, light, reproductive capacity, time post mortem, time post feeding, 
# exercise details, menstruation, personality, litter size 
#library.loc[library["condition"] == "old", "condition"] = "new"


# view
display_df(library)

#### annotator id, last modification date

In [47]:
library.loc[:,'annotatorId'] = 'SAC'
library.loc[:,'lastModificationDate'] = '2024-09-19'

# view
display_df(library)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate,library_contruction_protocol,source_qc,lib_name_2,lib_name_3,source_name,individual,infoStage_2,infoStage_3
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_102,SAMN27103190,,,,,,,,SAC,2024-09-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_100,SAMN27103191,,,,,,,,SAC,2024-09-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_98,SAMN27103192,,,,,,,,SAC,2024-09-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_95,SAMN27103193,,,,,,,,SAC,2024-09-19,"RNA extraction was achieved using Macherey Nagel Nucleospin RNA kit. RNA concentration was determined using Nanodrop and RNA quality control was checked using Agilent. A total amount of 1 μg RNA per sample was used as input material for the RNA sample preparations. Sequencing libraries were generated using NEBNext® UltraTM RNA Library Prep Kit for Illumina® (NEB, USA) following manufacturer's recommendations.",,,,Chorioallantoic membrane,,,
4,SRX14667189,SRP366684,Il

#### comments

In [ ]:
#library.loc[:,'comment'] = ''

#### save complete file with correct columns

In [48]:
library_file_complete = library[['#libraryId', 'experimentId', 'platform', 'SRSId', 'anatId', 'anatName', 'stageId', 'stageName', 'url_GSM', 'infoOrgan', 'infoStage', 'anatAnnotationStatus', 'anatBiologicalStatus', 'stageAnnotationStatus', 'sex', 'strain', 'genotype', 'speciesId', 'protocol', 'protocolType', 'RNASelection', 'globin_reduction', 'replicate', 'lib_name', 'sampleName', 'sampleAge_value', 'sampleAge_unit', 'PATOid', 'PATOname','comment', 'condition', 'physiologicalStatus', 'annotatorId', 'lastModificationDate']]

library_file_complete.to_csv(library_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

# view
display_df(library_file_complete)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate
0,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984242,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_102,SAMN27103190,,,,,,,,SAC,2024-09-19
1,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984241,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_100,SAMN27103191,,,,,,,,SAC,2024-09-19
2,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984240,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_98,SAMN27103192,,,,,,,,SAC,2024-09-19
3,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984239,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_95,SAMN27103193,,,,,,,,SAC,2024-09-19
4,SRX14667189,SRP366684,Illumina NovaSeq 6000,SRS12431810,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984238,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_93,SAMN27103194,,,,,,,,SAC,2024-09-19
5,SRX14667188,SRP366684,Illumina NovaSeq 6000,SRS12431812,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984237,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_88,SAMN27103195,,,,,,,,SAC,2024-09-19
6,SRX14667187,SRP366684,Illumina NovaSeq 6000,SRS12431807,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984236,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_86,SAMN27103196,,,,,,,,SAC,2024-09-19
7,SRX14667186,SRP366684,Illumina NovaSeq 6000,SRS12431808,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984235,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_83,SAMN27103197,,,,,,,,SAC,2024-09-19
8,SRX14667185,SRP366684,Illumina NovaSeq 6000,SRS12431809,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5984234,Chorioallantoic membrane,Prehat

### experiment annotations

In [49]:
experiment = pd.read_csv(experiment_path_from_script, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
display_df(experiment)

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,projectTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
0,SRP366684,"Transcriptome of the chicken chorioallantoic membrane from male and female embryos, after 11 and 15 days of incubation","The aim the the sequencing project was to investigate differentially expressed genes between fertilized eggs incubated during 11 or 15 days of incubation (EID11 and EID15). At each stage (EID11 and EID15), chorioallantoic membranes from 10 male embryos and 10 females embryos were collected and processed for RNA extraction and RNAseq analysis (n=40 in total). Thus, the design of the protocol also allowed comparison of differentially expressed genes in the chorioallantoic membrane between male and female embryos Overall design: mRNA profiles of the chorioallantoic membrane between 11 and 15 days of incubation (ROSS 308 broiler line)",SRA,,,,,,,PRJNA821480,36642281,,10.1016/j.ygeno.2023.110564,,


#### experiment and protocol details

In [53]:
# this will give you the number of rows in the complete library file 
# this should be the number of annotated libraries
ann_lib = len(library_file_complete.index)
len(library_file_complete.index)

40

In [54]:
# partial or total
experiment.loc[:,'experimentStatus'] = 'total'
# experiment.loc[:,'projectTags'] = '' 
# see above cell, also can add as free text
experiment.loc[:,'numberOfAnnotatedLibraries'] = ann_lib

# these variables should already exist from above but if not can just add as free text
experiment.loc[:,'protocol'] = my_protocol
experiment.loc[:,'protocolType'] = my_protocolType

display_df(experiment)

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,projectTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
0,SRP366684,"Transcriptome of the chicken chorioallantoic membrane from male and female embryos, after 11 and 15 days of incubation","The aim the the sequencing project was to investigate differentially expressed genes between fertilized eggs incubated during 11 or 15 days of incubation (EID11 and EID15). At each stage (EID11 and EID15), chorioallantoic membranes from 10 male embryos and 10 females embryos were collected and processed for RNA extraction and RNAseq analysis (n=40 in total). Thus, the design of the protocol also allowed comparison of differentially expressed genes in the chorioallantoic membrane between male and female embryos Overall design: mRNA profiles of the chorioallantoic membrane between 11 and 15 days of incubation (ROSS 308 broiler line)",SRA,total,,40,NEBNext Ultra RNA Library Prep Kit,full_length,,PRJNA821480,36642281,,10.1016/j.ygeno.2023.110564,,


#### paper and xrefs

In [55]:
experiment.loc[:,'GSE'] = 'GSE199780'
#experiment.loc[:,'Bioproject'] = '' 
#experiment.loc[:,'PMID'] = ''
experiment.loc[:,'reference_url'] = 'https://www.sciencedirect.com/science/article/pii/S0888754323000083?via%3Dihub'
#experiment.loc[:,'DOI'] = ''
# format here is ID[ID type] ex. 39263233[PMID]
# ID type options are: SRA (for SRP, ERP and DRP IDs), GEO, ArrayExpress, PMID, URL
experiment.loc[:,'xrefs'] = '39263233[PMID]'

display_df(experiment)

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,projectTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
0,SRP366684,"Transcriptome of the chicken chorioallantoic membrane from male and female embryos, after 11 and 15 days of incubation","The aim the the sequencing project was to investigate differentially expressed genes between fertilized eggs incubated during 11 or 15 days of incubation (EID11 and EID15). At each stage (EID11 and EID15), chorioallantoic membranes from 10 male embryos and 10 females embryos were collected and processed for RNA extraction and RNAseq analysis (n=40 in total). Thus, the design of the protocol also allowed comparison of differentially expressed genes in the chorioallantoic membrane between male and female embryos Overall design: mRNA profiles of the chorioallantoic membrane between 11 and 15 days of incubation (ROSS 308 broiler line)",SRA,total,,40,NEBNext Ultra RNA Library Prep Kit,full_length,GSE199780,PRJNA821480,36642281,https://www.sciencedirect.com/science/article/pii/S0888754323000083?via%3Dihub,10.1016/j.ygeno.2023.110564,39263233[PMID],


#### comments

In [ ]:
#experiment.loc[:,'comment'] = ''

display_df(experiment)

#### save complete file

In [56]:
experiment.to_csv(experiment_to_add_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)

### QA time

In [65]:
library_to_add = pd.read_csv(library_to_add_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
experiment_to_add = pd.read_csv(experiment_to_add_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)

#### to add things here

#### check columns match

In [67]:
# pull from git and pull in library/experiment file
! git pull
git_library = pd.read_csv(git_library_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)
git_experiment = pd.read_csv(git_experiment_path, sep='\t', index_col=False, keep_default_na=False, na_values=['NULL','null', 'nan','NaN'], dtype=object)

# library file
if set(library_to_add.columns) == set(git_library.columns):
    print('The columns in the library file match')
else:
    print('The columns in the library file DO NOT MATCH')

# experiment file
if set(experiment_to_add.columns) == set(git_experiment.columns):
    print('The columns in the experiment file match')
else:
    print('The columns in the experiment file DO NOT MATCH')


# maybe to make this something more like "COLUMNS GOOD - LIBRARY" and "COLUMNS BAD - EXPERIMENT"

Already up to date.
The columns in the library file match
The columns in the experiment file match


#### view files

In [68]:
library_git_plus_new = pd.concat([git_library, library_to_add], ignore_index = True, sort = False)
library_git_plus_new.tail(n=45)

,#libraryId,experimentId,platform,SRSId,anatId,anatName,stageId,stageName,url_GSM,infoOrgan,infoStage,anatAnnotationStatus,anatBiologicalStatus,stageAnnotationStatus,sex,strain,genotype,speciesId,protocol,protocolType,RNASelection,globin_reduction,replicate,lib_name,sampleName,sampleAge_value,sampleAge_unit,PATOid,PATOname,comment,condition,physiologicalStatus,annotatorId,lastModificationDate
38293,SRX3958636,SRP140736,Illumina HiSeq 4000,SRS3188951,UBERON:0001401,longissimus thoracis muscle,OariDv:0000004,suckling stage,,longissimus dorsi,newborn,perfect match,partial sampling,missing child term,M,Hu,,9940,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,C_TR_replicate 6,SAMN08934749,,,,,,control (no folic acid dietary supplement),,SAC,2024-09-18
38294,SRX3958635,SRP140736,Illumina HiSeq 4000,SRS3188949,UBERON:0001401,longissimus thoracis muscle,OariDv:0000004,suckling stage,,longissimus dorsi,newborn,perfect match,partial sampling,missing child term,M,Hu,,9940,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,F32_TR_replicate 3,SAMN08934758,,,,,,folate 2 (dietary supplement 32 mg folic acid ...,,SAC,2024-09-18
38295,SRX3958634,SRP140736,Illumina HiSeq 4000,SRS3188953,UBERON:0001401,longissimus thoracis muscle,OariDv:0000004,suckling stage,,longissimus dorsi,newborn,perfect match,partial sampling,missing child term,M,Hu,,9940,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,F16_TR_replicate 1,SAMN08934750,,,,,,folate 1 (dietary supplement 16 mg folic acid ...,,SAC,2024-09-18
38296,SRX3958633,SRP140736,Illumina HiSeq 4000,SRS3188952,UBERON:0001401,longissimus thoracis muscle,OariDv:0000004,suckling stage,,longissimus dorsi,newborn,perfect match,partial sampling,missing child term,F,Hu,,9940,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,F32_TR_replicate 4,SAMN08934759,,,,,,folate 2 (dietary supplement 32 mg folic acid ...,,SAC,2024-09-18
38297,SRX3958632,SRP140736,Illumina HiSeq 4000,SRS3188950,UBERON:0001401,longissimus thoracis muscle,OariDv:0000004,suckling stage,,longissimus dorsi,newborn,perfect match,partial sampling,missing child term,M,Hu,,9940,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,F16_TR_replicate 2,SAMN08934751,,,,,,folate 1 (dietary supplement 16 mg folic acid ...,,SAC,2024-09-18
38298,SRX14667193,SRP366684,Illumina NovaSeq 6000,SRS12431814,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_102,SAMN27103190,,,,,,,,SAC,2024-09-19
38299,SRX14667192,SRP366684,Illumina NovaSeq 6000,SRS12431815,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_100,SAMN27103191,,,,,,,,SAC,2024-09-19
38300,SRX14667191,SRP366684,Illumina NovaSeq 6000,SRS12431813,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,M,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15M_98,SAMN27103192,,,,,,,,SAC,2024-09-19
38301,SRX14667190,SRP366684,Illumina NovaSeq 6000,SRS12431811,UBERON:0005335,chorioallantoic membrane,GgalDv:0000055,Hamburger Hamilton stage 41,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,Chorioallantoic membrane,Prehatch : 15 days of incubation,perfect match,not documented,perfect match,F,Ross 308,,9031,NEBNext Ultra RNA Library Prep Kit,full_length,polyA,,,CAM15F_95,SAMN27103193,,,,,,,,SAC,2024-09-19
38302,SRX14667189,SRP366684,Illumina NovaSeq 6000,SRS12431810,UBERON:00053

In [69]:
experiment_git_plus_new = pd.concat([git_experiment, experiment_to_add], ignore_index = True, sort = False)
experiment_git_plus_new.tail(n=5)

,#experimentId,experimentName,experimentDescription,experimentSource,experimentStatus,projectTags,numberOfAnnotatedLibraries,protocol,protocolType,GSE,Bioproject,PMID,reference_url,DOI,xrefs,comment
771,SRP177954,goat liver data,Ureagenesis plays a significant role in the ho...,SRA,total,FAANG,18,NEB Next Ultra RNA Library Prep Kit,full_length,,PRJNA512146,31726987,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,10.1186/s12864-019-6233-9,,2 diets conditions that look acceptable
772,SRP474581,Pituitary transcriptome data of Leizhou goats ...,Pituitary transcriptome data of Leizhou goats ...,SRA,total,,7,"Ribo-Zero rRNA Removal Kit (Illumina, Inc.) an...",full_length,,PRJNA1043736,38152654,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,10.3389/fgene.2023.1303031,,
773,SRP329780,Wu'an goat muscle sequencing data (mRNA+lncRNA),Sequencing data of the longissimus dorsi muscl...,SRA,total,,6,Ribo-zeroTM Gole Kit,full_length,,PRJNA749569,36230427,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,10.3390/ani12192683,,no brand protocol named
774,SRP140736,Ovis aries Transcriptome or Gene expression,folate supplentation on the effects of muscle ...,SRA,total,FAANG,18,NEBNext Ultra RNA Library Prep Kit,full_length,,PRJNA450309,31229912,https://www.sciencedirect.com/science/article/...,10.1016/j.jnutbio.2019.05.011,,
775,SRP366684,Transcriptome of the chicken chorioallantoic m...,The aim the the sequencing project was to inve...,SRA,total,,40,NEBNext Ultra RNA Library Prep Kit,full_length,GSE199780,PRJNA821480,36642281,https://www.sciencedirect.com/science/article/...,10.1016/j.ygeno.2023.110564,39263233[PMID],


### add annotations to git

In [70]:
! git pull

Already up to date.


In [73]:
library_git_plus_new.to_csv(git_library_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
experiment_git_plus_new.to_csv(git_experiment_path, sep="\t", index=False, quoting=csv.QUOTE_ALL)
update_format(git_library_path)
update_format(git_experiment_path)

In [74]:
## if you don't see the notebook here it's because you need to save it
! git status

On branch develop
Your branch is up to date with 'origin/develop'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../1_sara_bulk_template.ipynb
	modified:   ../../../RNA_Seq/RNASeqExperiment.tsv
	modified:   ../../../RNA_Seq/RNASeqLibrary.tsv

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	./

no changes added to commit (use "git add" and/or "git commit -a")


In [77]:
! git add $git_experiment_path $git_library_path $script_file $path_to_output

In [78]:
! git status

On branch develop
Your branch is up to date with 'origin/develop'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   RNASeqExperiment_SRP366684.tsv
	new file:   RNASeqLibrary_SRP366684.tsv
	new file:   SRP366684.ipynb
	new file:   complete_RNASeqExperiment_SRP366684.tsv
	new file:   complete_RNASeqLibrary_SRP366684.tsv
	modified:   ../../../RNA_Seq/RNASeqExperiment.tsv
	modified:   ../../../RNA_Seq/RNASeqLibrary.tsv

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../1_sara_bulk_template.ipynb



In [79]:
! git commit -m $commit_message_exp

[develop 8ef0591] adding annotated bulk experiment SRP366684 and adding annotation files to notebook folder
 7 files changed, 18173 insertions(+)
 create mode 100644 Notebooks/bulk/SRP366684/RNASeqExperiment_SRP366684.tsv
 create mode 100644 Notebooks/bulk/SRP366684/RNASeqLibrary_SRP366684.tsv
 create mode 100644 Notebooks/bulk/SRP366684/SRP366684.ipynb
 create mode 100644 Notebooks/bulk/SRP366684/complete_RNASeqExperiment_SRP366684.tsv
 create mode 100644 Notebooks/bulk/SRP366684/complete_RNASeqLibrary_SRP366684.tsv


In [80]:
! git push

Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 12 threads
Compressing objects: 100% (12/12), done.
Writing objects: 100% (13/13), 114.51 KiB | 8.81 MiB/s, done.
Total 13 (delta 6), reused 0 (delta 0), pack-reused 0
remote: 
remote: To create a merge request for develop, visit:
remote:   https://gitlab.sib.swiss/Bgee/expression-annotations/-/merge_requests/new?merge_request%5Bsource_branch%5D=develop
remote: 
To https://gitlab.sib.swiss/Bgee/expression-annotations.git
   4584661..8ef0591  develop -> develop
